## Collaborative Filtering with Cold Start
### Model inspried by: Ibtesam Ahmed
#### Link: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook


##### Using SVD learning model and including user cold start scenario
##### Base Model

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy

In [ ]:
# Load dataset
ratings = pd.read_csv('../tmdb/ratings_small.csv')
reader = Reader(rating_scale=(0.5, 5.0))

In [ ]:
# Choose a user to simulate cold start
user_id = 1
user_ratings = ratings[ratings['userId'] == user_id]

# Keep only 5 ratings for the user, remove the rest
keep_ratings = user_ratings.sample(n=5, random_state=42)   # the 5 ratings we keep
remove_ratings = user_ratings.drop(keep_ratings.index)      # the rest we remove

# Remove the other ratings from the dataset for training
train_ratings = ratings.drop(remove_ratings.index)

In [ ]:
# Print # of movies user     has rated
print(len(user_ratings))

In [ ]:
# Load the training set into Surprise
train_data = Dataset.load_from_df(train_ratings[['userId','movieId','rating']], reader)
trainset = train_data.build_full_trainset()

In [ ]:
# Train the SVD model
svd = SVD()
svd.fit(trainset)

In [ ]:
# Predict the removed ratings for the user
predictions = []
for movie_id in remove_ratings['movieId']:
    true_rating = ratings[(ratings['userId'] == user_id) & (ratings['movieId'] == movie_id)]['rating'].values[0]
    pred = svd.predict(user_id, movie_id, r_ui=true_rating)
    predictions.append(pred)
    print(pred)

In [ ]:
# Evaluate performance on these predictions
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)